# Initial Setup

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 152.9 MB 34 kB/s 
     |████████████████████████████████| 57 kB 2.5 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.307 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [ ]:
!pip install transformers


     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install Sentencepiece


     |████████████████████████████████| 1.2 MB 4.0 MB/s 


In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
device = xm.xla_device()

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaModel
from transformers import AdamW

In [ ]:
rTokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [ ]:
print(rTokenizer.vocab_size)

250002


# Data

## Import


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1RA8uE4DRgEV2iHx2kLnPvUgXWZ3gTJM1' -O 'chinese.csv'
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1f2He-xBuCjW4BU1BqWKXivCE1_RhSGha' -O 'english.csv'
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13OYXerLI7O3kGfanfoBlcEFqvGzi9Q0z' -O 'spanish.csv'
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t-cPh9CBtKZYTDzunGQhsFbIYWfyRGeQ' -O 'marathi.csv'
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Jrpm7fMu2nb0axD0iiJ8mgDoyiVLvixc' -O 'urdu.tsv'


--2022-05-13 14:45:41--  https://docs.google.com/uc?export=download&id=1RA8uE4DRgEV2iHx2kLnPvUgXWZ3gTJM1
Resolving docs.google.com (docs.google.com)... 173.194.192.100, 173.194.192.139, 173.194.192.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.192.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/htv1pivoeto0ep64gatpapeppv85juhg/1652453100000/08219369989881812643/*/1RA8uE4DRgEV2iHx2kLnPvUgXWZ3gTJM1?e=download [following]
--2022-05-13 14:45:44--  https://doc-14-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/htv1pivoeto0ep64gatpapeppv85juhg/1652453100000/08219369989881812643/*/1RA8uE4DRgEV2iHx2kLnPvUgXWZ3gTJM1?e=download
Resolving doc-14-6k-docs.googleusercontent.com (doc-14-6k-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-14-6k-docs.googleusercontent.com (doc-14-6k-docs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import gc

In [ ]:
urduRaw = pd.read_csv("urdu.tsv", sep='\t')
spanishRaw = pd.read_csv("spanish.csv")
marathiRaw = pd.read_csv("marathi.csv")
chineseRaw = pd.read_csv("chinese.csv")

In [ ]:
# raw data file too large for !wget. Imported from google drive
englishRaw = pd.read_csv("drive/MyDrive/dataset/english.csv", delimiter=',', header=None)

## Cleaning
### Spanish

In [ ]:
spanish = spanishRaw[["text", "airline_sentiment"]].copy(deep=True).sample(frac=1).reset_index(drop=True)
spanish.rename(columns = {'airline_sentiment':'label'}, inplace = True)
spanish = spanish[spanish['label']!="neutral"]
spanish.loc[(spanish.label == "negative"), "label"] = 0
spanish.loc[(spanish.label == 'positive'), "label"] = 1
spanish['language']='spanish'

In [ ]:
spanish['label'].value_counts()

0    3769
1    1489
Name: label, dtype: int64

In [ ]:
val_sizeS = int(spanish.shape[0] * 0.15)
test_sizeS = int(spanish.shape[0] * 0.15)
test_spanish = spanish[:test_sizeS].reset_index(drop=True)
val_spanish = spanish[test_sizeS:test_sizeS+val_sizeS].reset_index(drop=True)
train_spanish = spanish[test_sizeS+val_sizeS:].reset_index(drop=True)

### Urdu

In [ ]:
urdu=urduRaw.copy(deep=True).sample(frac=1).reset_index(drop=True)
urdu.rename(columns = {'Tweet':'text','Class':'label'}, inplace = True)
urdu = urdu[urdu['label'].notna()]
urdu = urdu[urdu['label']!='O']
urdu.loc[(urdu.label == "N"), "label"] = 0
urdu.loc[(urdu.label == 'P'), "label"] = 1
urdu['language']='urdu'

In [ ]:
urdu['label'].value_counts()

0    499
1    480
Name: label, dtype: int64

In [ ]:
val_sizeU = int(urdu.shape[0] * 0.15)
test_sizeU = int(urdu.shape[0] * 0.15)
test_urdu = urdu[:test_sizeU].reset_index(drop=True)
val_urdu = urdu[test_sizeU:test_sizeU+val_sizeU].reset_index(drop=True)
train_urdu = urdu[test_sizeU+val_sizeU:].reset_index(drop=True)

### Marathi

In [ ]:
marathi=marathiRaw.copy(deep=True).sample(frac=1).reset_index(drop=True)
marathi.rename(columns = {'tweet':'text'}, inplace = True)
marathi = marathi[marathi['label']!=0]
marathi.loc[(marathi.label == -1), "label"] = 0
marathi['language']='marathi'

In [ ]:
marathi['label'].value_counts()

0    4038
1    4038
Name: label, dtype: int64

In [ ]:
val_sizeM = int(marathi.shape[0] * 0.15)
test_sizeM = int(marathi.shape[0] * 0.15)
test_marathi = marathi[:test_sizeM].reset_index(drop=True)
val_marathi = marathi[test_sizeM:test_sizeM+val_sizeM].reset_index(drop=True)
train_marathi = marathi[test_sizeM+val_sizeM:].reset_index(drop=True)

### Chinese

In [ ]:
chinese = chineseRaw[["review", "label"]].copy(deep=True).sample(frac=1).reset_index(drop=True)
chinese.rename(columns = {'review':'text'}, inplace = True)

In [ ]:
chinese['language']='chinese'

In [ ]:
chinese = chinese.sample(9000).reset_index(drop=True).copy()
chinese['label'].value_counts()

1    4530
0    4470
Name: label, dtype: int64

In [ ]:
val_sizeC = int(chinese.shape[0] * 0.15)
test_sizeC = int(chinese.shape[0] * 0.15)
test_chinese = chinese[:test_sizeC].reset_index(drop=True)
val_chinese = chinese[test_sizeC:test_sizeC+val_sizeC].reset_index(drop=True)
train_chinese = chinese[test_sizeC+val_sizeC:].reset_index(drop=True)

### English

In [ ]:
english = englishRaw[[5, 0]].copy(deep=True).sample(frac=1).reset_index(drop=True)
english.rename(columns = {5:'text', 0:'label'}, inplace = True)
english.loc[(english.label == 4), "label"] = 1
english['language']='english'

In [ ]:
english = english.sample(9000).reset_index(drop=True).copy()
english['label'].value_counts()

1    4536
0    4464
Name: label, dtype: int64

In [ ]:
val_sizeE = int(english.shape[0] * 0.15)
test_sizeE = int(english.shape[0] * 0.15)
test_english = english[:test_sizeE].reset_index(drop=True)
val_english = english[test_sizeE:test_sizeE+val_sizeE].reset_index(drop=True)
train_english = english[test_sizeE+val_sizeE:].reset_index(drop=True)

In [ ]:
pd.concat([chinese[0:1],english[0:1],spanish[0:1],marathi[0:1],urdu[0:1]])

,text,label,language
0,忍不住再转一下！@LR68 @泡泡_860409 //@Tsnow-Queen: [给力][...,1,chinese
0,its funny when you put everything you have int...,0,english
0,Retrasamos el hasta luego España porque el avi...,0,spanish
1,"उत्तर मुंबईच्या वस्त्यां, गल्ली गल्लीमधे गोप...",0,marathi
0,چندے سے انقلاب اور تھُک میں پکوڑے تلنا,0,urdu


## Combine and Summary

In [ ]:
test = pd.concat([test_english,test_chinese,test_urdu, test_spanish, test_marathi]).sample(frac=1).reset_index(drop=True)
val = pd.concat([val_english,val_chinese,val_urdu, val_spanish, val_marathi]).sample(frac=1).reset_index(drop=True)
train = pd.concat([train_english,train_chinese,train_urdu, train_spanish, train_marathi]).sample(frac=1).reset_index(drop=True)

In [ ]:
testSummary = pd.DataFrame(columns = test['language'].unique())
temp=[0,0,0,0,0]
testSummary.loc['positive']=temp
testSummary.loc['negative']=temp
testSummary.loc['total']=temp
for lang in test['language'].unique():
  pos=test.loc[(test.language == lang)]['label'].value_counts()[1]
  neg=test.loc[(test.language == lang)]['label'].value_counts()[0]
  testSummary.loc['positive'][lang]=pos
  testSummary.loc['negative'][lang]=neg
  testSummary.loc['total'][lang]=pos+neg
testSummary['total'] = testSummary.sum(axis=1)
testSummary

,spanish,chinese,english,marathi,urdu,total
positive,228,659,675,605,72,2239.0
negative,560,691,675,606,74,2606.0
total,788,1350,1350,1211,146,4845.0


In [ ]:
valSummary = pd.DataFrame(columns = val['language'].unique())
temp=[0,0,0,0,0]
valSummary.loc['positive']=temp
valSummary.loc['negative']=temp
valSummary.loc['total']=temp
for lang in val['language'].unique():
  pos=val.loc[(val.language == lang)]['label'].value_counts()[1]
  neg=val.loc[(val.language == lang)]['label'].value_counts()[0]
  valSummary.loc['positive'][lang]=pos
  valSummary.loc['negative'][lang]=neg
  valSummary.loc['total'][lang]=pos+neg
valSummary['total'] = valSummary.sum(axis=1)
valSummary

,chinese,english,marathi,spanish,urdu,total
positive,697,699,585,241,74,2296.0
negative,653,651,626,547,72,2549.0
total,1350,1350,1211,788,146,4845.0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
trainSummary = pd.DataFrame(columns = train['language'].unique())
temp=[0,0,0,0,0]
trainSummary.loc['positive']=temp
trainSummary.loc['negative']=temp
trainSummary.loc['total']=temp
for lang in train['language'].unique():
  pos=train.loc[(train.language == lang)]['label'].value_counts()[1]
  neg=train.loc[(train.language == lang)]['label'].value_counts()[0]
  trainSummary.loc['positive'][lang]=pos
  trainSummary.loc['negative'][lang]=neg
  trainSummary.loc['total'][lang]=pos+neg
trainSummary['total'] = trainSummary.sum(axis=1)
trainSummary

,english,chinese,spanish,urdu,marathi,total
positive,3162,3174,1020,334,2848,10538.0
negative,3138,3126,2662,353,2806,12085.0
total,6300,6300,3682,687,5654,22623.0


In [ ]:
test_texts, test_labels = list(test.text), list(test.label)
val_texts, val_labels = list(val.text), list(val.label)
train_texts, train_labels = list(train.text), list(train.label)

# Model Setup

## Define parameters

In [ ]:
# Parameters for fine-tuning
L_RATE = 3e-5
MAX_LEN = 20
NUM_EPOCHS = 3
BATCH_SIZE = 64
NUM_CORES = os.cpu_count()
NUM_CORES

2

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    'xlm-roberta-base', 
    num_labels = 2,
)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

## Tokenization

In [ ]:
class CompDataset(Dataset):
  
    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        text = self.df_data.loc[index, 'text']
        # tokenization
        encoded_dict = rTokenizer.encode_plus(
                    text,
                    max_length = MAX_LEN,
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt',
               )
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, 'label'])
        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
test_data = CompDataset(test)
train_data = CompDataset(train)
val_data = CompDataset(val)

In [ ]:
print(len(test_data))
print(len(train_data))
print(len(val_data))

4845
22623
4845


# Baseline Results

In [ ]:
def createBaseline(data, name):
    test_data = CompDataset(data)
    dataloader = torch.utils.data.DataLoader(test_data,
                      batch_size=BATCH_SIZE,
                      shuffle=True,
                      num_workers=NUM_CORES)
    model.eval()
    torch.set_grad_enabled(False)
    targets_list=[]
    for j, batch in enumerate(dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0: 
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(name,' test acc: ', acc)
    print(name, ': test F1: ', f1)

In [ ]:
createBaseline(test_chinese, 'Chinese')
createBaseline(test_english, 'English')
createBaseline(test_urdu, 'Urdu')
createBaseline(test_marathi, 'Marathi')
createBaseline(test_spanish, 'Spanish')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Chinese  test acc:  0.5118518518518519
Chinese : test F1:  0.0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

English  test acc:  0.49925925925925924
English : test F1:  0.0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Urdu  test acc:  0.5136986301369864
Urdu : test F1:  0.0273972602739726


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

Marathi  test acc:  0.4995871180842279
Marathi : test F1:  0.0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Spanish  test acc:  0.7106598984771574
Spanish : test F1:  0.0


# Fine-tune

In [ ]:
import random
import gc


In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7fda97f9ee50>


In [ ]:
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
train_data = CompDataset(train_marathi)
val_data = CompDataset(val)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=NUM_CORES)
val_dataloader = torch.utils.data.DataLoader(val_data,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=NUM_CORES)

In [ ]:
gc.collect()

88

In [ ]:
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []

for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)
    total_train_loss = 0
    gc.collect()

    for i, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        loss = outputs[0]
        total_train_loss = total_train_loss + loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        xm.optimizer_step(optimizer, barrier=True)   
        gc.collect()

    print('Train loss:' ,total_train_loss)

    gc.collect()

    print('\nValidation...')
    model.eval()
    torch.set_grad_enabled(False)
    total_val_loss = 0
    for j, batch in enumerate(val_dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        total_val_loss = total_val_loss + loss.item()
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0:  # first batch
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
        gc.collect()
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    val_acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('Val f1:' ,f1)
    gc.collect()

    torch.save(model.state_dict(), 'model.pt')

    gc.collect()

        


======== Epoch 1 / 3 ========
Training...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Train loss: 49.59836035966873

Validation...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Val loss: 55.194731175899506
Val acc:  0.6825593395252838
Val f1: 0.5723025583982202

======== Epoch 2 / 3 ========
Training...


...

Val loss: 52.326828330755234
Val acc:  0.7023735810113519
Val f1: 0.7134340222575516

======== Epoch 3 / 3 ========
Training...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Train loss: 32.779386192560196

Validation...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

Val loss: 55.69235819578171
Val acc:  0.7027863777089783
Val f1: 0.7011207970112079


In [ ]:
gc.collect()

43

# Result and Anlaysis

In [ ]:

def singleLangTest(data, name):
    test_data = CompDataset(data)
    dataloader = torch.utils.data.DataLoader(test_data,
                      batch_size=BATCH_SIZE,
                      shuffle=True,
                      num_workers=NUM_CORES)
    model.eval()
    torch.set_grad_enabled(False)
    targets_list=[]
    for j, batch in enumerate(dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0: 
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    # print(name)
    print(name,'test acc: ', acc)
    print(name,'test F1: ', f1)
    print(name,'precision ', precision)
    print(name,'recall ', recall)


In [ ]:
singleLangTest(test_chinese,'chinese')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

chinese test acc:  0.6881481481481482
chinese test F1:  0.6902133922001472
chinese precision  0.67
chinese recall  0.7116843702579666


In [ ]:
singleLangTest(test_english,'english')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

english test acc:  0.6422222222222222
english test F1:  0.6403574087862994
english precision  0.6437125748502994
english recall  0.6370370370370371


In [ ]:
singleLangTest(test_spanish,'spanish')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

spanish test acc:  0.7106598984771574
spanish test F1:  0.6055363321799309
spanish precision  0.5
spanish recall  0.7675438596491229


In [ ]:
singleLangTest(test_urdu,'urdu')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

urdu test acc:  0.6095890410958904
urdu test F1:  0.583941605839416
urdu precision  0.6153846153846154
urdu recall  0.5555555555555556


In [ ]:
singleLangTest(test_marathi,'marathi')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

marathi test acc:  0.8232865400495458
marathi test F1:  0.8348765432098765
marathi precision  0.7829232995658466
marathi recall  0.8942148760330578
